# Comparison of NA Screening Results from Multiple CSV Files

This notebook processes one or more CSV files (each representing data from the same site at different times) using MiBiPreT functions. It loads, standardizes, and analyzes each CSV file, and then extracts only the following columns for comparison:

- sample_nr
- obs_well
- na_traffic_light
- intervention_traffic
- intervention_number
- intervention_contaminants

The filtered results from each file are then combined side by side.

## 1. Import Required Libraries

The following cell imports all necessary modules and libraries.

In [1]:
import mibipret.analysis.sample.screening_NA as na
from mibipret.data.check_data import standardize
from mibipret.data.load_data import load_csv
from IPython.display import display
import pandas as pd

## 2. Define CSV File Paths

List the file paths for the CSV files to be compared. Modify the list below with your actual file paths.

In [2]:
file_paths = [
    "../data/cleaned/na_screening/cw_T0_BTEXN.csv",
    "../data/cleaned/na_screening/cw_T1_BTEXN.csv",
    "../data/cleaned/na_screening/cw_T2_BTEXN.csv"
]

## 3. Ensure File Paths is a List

If a single file path is provided as a string, it is converted into a list to ensure uniform processing.

In [3]:
if isinstance(file_paths, str):
    file_paths = [file_paths]

## 4. Initialize Results Dictionary

Create an empty dictionary to store the filtered NA screening results from each CSV file.

In [4]:
results = {}

## 5. Process Each CSV File

Loop through each CSV file: load the data, standardize it, run the screening analysis, and extract the selected columns for comparison.

In [5]:
selected_columns = [
    "sample_nr", 
    "obs_well", 
    "na_traffic_light", 
    "intervention_traffic", 
    "intervention_number", 
    "intervention_contaminants"
]

for file in file_paths:
    print(f"Processing file: {file}")
    
    # Load raw data and units from the CSV file
    data_raw, units = load_csv(file, verbose=True)
    
    # Standardize the data (reducing to known quantities and cleaning values)
    data, units = standardize(data_raw, reduce=True, verbose=True)
    
    # Optional: additional calculations can be performed here (e.g., tot_redct, tot_oxi, etc.)
    tot_redct = na.reductors(data, verbose=True, ea_group="ONS")
    tot_oxi = na.oxidators(data, verbose=True, contaminant_group="BTEX")
    e_bal = na.electron_balance(data, verbose=True)
    na_traffic = na.NA_traffic(data, verbose=True)
    tot_cont = na.total_contaminant_concentration(data, verbose=True, contaminant_group="BTEX")
    na_interventation = na.thresholds_for_intervention(data, verbose=True, contaminant_group="BTEX")
    
    # Get the final NA screening table for this dataset
    data_na = na.screening_NA(data)
    
    # Extract only the selected columns for comparison
    data_na_filtered = data_na[selected_columns]
    
    # Store the filtered result using the file name as key
    results[file] = data_na_filtered

Processing file: ../data/cleaned/na_screening/cw_T0_BTEXN.csv
 Running function 'load_csv()' on data file  ../data/cleaned/na_screening/cw_T0_BTEXN.csv
Units of quantities:
-------------------
  sample_nr obs_well Benzene Toluene Ethylbenzene O Xylene P/M Xylene  \
0       NaN      NaN    ug/l    ug/l         ug/l     ug/l       ug/l   

  Naphthalene depth  pH    O2 Redox Fe II    Mn chloride nitrite Nitrite - N  \
0        ug/l     m NaN  mg/l    mV  mg/l  mg/l     mg/l    mg/l        mg/l   

  nitrate nitrate - N sulfate  
0    mg/l        mg/l    mg/l  
________________________________________________________________
Loaded data as pandas DataFrame:
--------------------------------
      sample_nr     obs_well Benzene Toluene Ethylbenzene O Xylene P/M Xylene  \
0           NaN          NaN    ug/l    ug/l         ug/l     ug/l       ug/l   
1    NL_CW_W_01  CW1_EFF-1-1     150      21          110       43         35   
2    NL_CW_W_02  CW1MF09-1-2     110       2           44    

## 6. Display the Comparison Table

If multiple CSV files have been processed, combine the filtered results side by side using `pd.concat`. If only one file is processed, display its result directly.

In [6]:
if len(results) == 1:
    display(list(results.values())[0])
else:
    # Concatenate the filtered screening results along columns
    comparison_table = pd.concat(results, axis=1)
    display(comparison_table)

../data/cleaned/na_screening/cw_T0_BTEXN.csv                                \
                                      sample_nr     obs_well na_traffic_light   
1                                    NL_CW_W_01  CW1_EFF-1-1            green   
2                                    NL_CW_W_02  CW1MF09-1-2            green   
3                                    NL_CW_W_03  CW1MF10-1-2            green   
4                                    NL_CW_W_04  CW1MF06-1-1              red   
5                                    NL_CW_W_05      INF-1-1            green   
6                                    NL_CW_W_06  CW1MF01-1-2              red   
7                                    NL_CW_W_07  CW1MF02-1-2            green   
8                                    NL_CW_W_08  CW1MF05-1-1            green   
9                                    NL_CW_W_09  CW2MF05-1-2            green   
10                                  NL_CW_W_010  CW2MF06-1-1            green   
11                                  NL_CW_W_011  CW2_EFF-1-1            green   
12                                  NL_CW_W_012  CW2MF01-1-1            green   
13                                  NL_CW_W_013  CW2MF02-1-1            green   
14                                  NL_CW_W_014  CW2MF09-1-1            green   
15                                  NL_CW_W_015  CW2MF10-1-1            green   
16                                  NL_CW_W_016  CW3_EFF-1-1            green   
17                                  NL_CW_W_017  CW3MF01-1-1            green   
18                                  NL_CW_W_018  CW3MF02-1-1            green   
19                                  NL_CW_W_019  CW3MF05-1-1            green   
20                                  NL_CW_W_020  CW3MF06-1-1            green   
21                                  NL_CW_W_021  CW3MF09-1-1            green   
22                                  NL_CW_W_022  CW3MF10-1-1            green   
23                                  NL_CW_W_023  CW2_EFF-1-2            green   

                                             \
   intervention_traffic intervention_number   
1                   red                   2   
2                   red                   1   
3                   red                   5   
4                   red                   1   
5                   red                   5   
6                   red                   1   
7                   red                   5   
8                 green                   0   
9                   red                   5   
10                  red                   5   
11                  red                   5   
12                  red                   1   
13                  red                   5   
14                  red                   1   
15                  red                   5   
16                  red                   5   
17                  red                   1   
18                  red                   5   
19                green                   0   
20                  red                   5   
21                green                   0   
22                  red                   5   
23                green                   0   

                                                       \
                            intervention_contaminants   
1                              [benzene, naphthalene]   
2                                           [benzene]   
3   [benzene, ethylbenzene, pm_xylene, o_xylene, n...   
4                                           [benzene]   
5   [benzene, ethylbenzene, pm_xylene, o_xylene, n...   
6                                           [benzene]   
7   [benzene, ethylbenzene, pm_xylene, o_xylene, n...   
8                                                  []   
9   [benzene, ethylbenzene, pm_xylene, o_xylene, n...   
10  [benzene, ethylbenzene, pm_xylene, o_xylene, n...   
11  [benzene, ethylbenzene, pm_xylene, o_xylene, n...   
12                                          [benzene]   
13 